In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="gtukr3MWulF7tRzHjKLr")
project = rf.workspace("inatel").project("pneumonia-classification-imrcv")
version = project.version(1)
dataset = version.download("folder")



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to pneumonia-classification-1 in folder:: 100%|██████████| 5196/5196 [00:20<00:00, 248.27it/s]


In [2]:
import os
os.listdir()

['Cardiovascular_Disease_Prediction_(AUC_=_0_93)_🩺.ipynb',
 'cell-images-for-detecting-malaria3.ipynb',
 'Chest_XRay (1).ipynb',
 'doctory_back',
 'is-diabetic.ipynb',
 'pneumonia-classification-1']

In [3]:
import os, shutil, random

base_dir = "pneumonia-classification-1/train"
split_dir = "split_dataset"



train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

os.makedirs(split_dir, exist_ok=True)
for cls in os.listdir(base_dir):
    cls_dir = os.path.join(base_dir, cls)
    imgs = os.listdir(cls_dir)
    random.shuffle(imgs)

    n_total = len(imgs)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)

    splits = {
        "train": imgs[:n_train],
        "val": imgs[n_train:n_train+n_val],
        "test": imgs[n_train+n_val:]
    }

    for split_name, files in splits.items():
        split_cls_dir = os.path.join(split_dir, split_name, cls)
        os.makedirs(split_cls_dir, exist_ok=True)
        for f in files:
            shutil.copy(os.path.join(cls_dir, f), os.path.join(split_cls_dir, f))


In [ ]:
!pip install ultralytics


In [ ]:
from ultralytics import YOLO


model = YOLO("yolo11n-cls.pt")


results = model.train(
    data="split_dataset",
    epochs=50,
    imgsz=224,
    batch=16,
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=split_dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0,

In [ ]:
from google.colab import files
files.download('/content/runs/classify/train/weights/best.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

shutil.make_archive('chest_xray_project', 'zip', '/content/pneumonia-classification-1')

'/content/chest_xray_project.zip'

In [ ]:
from google.colab import files
files.download('chest_xray_project.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from ultralytics import YOLO

# 1. Load the best model u got
model = YOLO("/content/runs/classify/train/weights/best.pt")


results = model.predict(source="/content/split_dataset/test/Normal/IM-0370-0001_jpeg.rf.f958c919f74e66e1a37fbfd72ee39e3e.jpg", show=True)


for r in results:
    print(f"Predicted class: {r.names[r.probs.top1]} ({r.probs.top1conf*100:.2f}% confidence)")



image 1/1 /content/split_dataset/test/Normal/IM-0370-0001_jpeg.rf.f958c919f74e66e1a37fbfd72ee39e3e.jpg: 224x224 Normal 0.99, Pneumonia_virus 0.01, Pneumonia_bacteria 0.00, 4.9ms
Speed: 6.6ms preprocess, 4.9ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Predicted class: Normal (99.24% confidence)


In [ ]:
# 2. This is the additional line you need to export to ONNX
# The file will be saved as 'best.onnx' in the 'runs/classify/export' folder
path_to_onnx = model.export(format="onnx")

print(f"ONNX model saved to: {path_to_onnx}")